<a href="https://colab.research.google.com/github/profteachkids/CHE5136_Fall2021/blob/main/Parameterize_FlattenUnknowns.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import jax.numpy as jnp
import jax

In [ ]:
def flat(unks):
    return np.concatenate([unk.pack() for unk in unks])

In [ ]:
class Unk():

    def __init__(self,x):
        self.x=x
        if type(x) is np.ndarray:
            self.shape=x.shape
            self.size = x.size
        else:
            self.shape=tuple()
            self.size=1

    def unpack(self,val):
        return val.reshape(self.shape)

    def pack(self):
        return np.ravel(self.x)

In [ ]:
def parameterize2(f, **kw):

    params={}
    vars=[]
    unks=[]
    for k,v in kw.items():
        if isinstance(v,Unk):
            vars.append(k)
            unks.append(v)
        else:
            params[k]=v
    indices=np.cumsum([unk.size for unk in unks])

    def unflat(v):
        return {k:unk.unpack(val) for k,unk,val in zip(vars,unks, np.split(v,indices))}

    def f_parameterized(v):
        var_dict = unflat(v)
        params.update(var_dict)
        return eqs(**params)


    return f_parameterized, unflat, params

In [ ]:
def eqs(vtot, mtot, x1, x2, P):
    return (x1+3*x2)+2*P

x1guess = Unk(np.array([0.1,0.9]))
x2guess = Unk(np.array([0.3,0.7]))
Pguess = Unk(1e5)

v=flat([x1guess, x2guess, Pguess])
f, unflat, params = parameterize2(eqs, x1=x1guess, x2=x2guess, P=Pguess, vtot=10., mtot=2.)

In [ ]:
unflat(v)

{'P': array(100000.), 'x1': array([0.1, 0.9]), 'x2': array([0.3, 0.7])}

In [ ]:
jax_f=jax.jit(f)

In [ ]:
jax_f(v)

DeviceArray([200001., 200003.], dtype=float32)

In [ ]:
jax.jacobian(f)(v)

DeviceArray([[1., 0., 3., 0., 2.],
             [0., 1., 0., 3., 2.]], dtype=float32)